In [7]:
import utils
import matplotlib.pyplot as plt
import pandas
import datetime
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
datafile = 'preprocessed_data.csv'

# train_X = []
# train_y = []
# step = 3
# end = 2019 - step
# for year in range(2010, end):
#     X, y = utils.get_data(year, year + step, year + step, datafile)
#     train_X.append(X)
#     train_y.append(y)

# print(train_X)
# print(train_y)

# model = SARIMAX(train_X, order = (), seasonal_order = ())

def forecastSARIMA(history, hyperparams):
    order, seasonal, trend = hyperparams
    model = SARIMAX(history, order = order, seasonal_order = seasonal, trend = trend, enforce_stationarity = False, enforce_invertibility = False)
    model_fit = model.fit(disp = False)
    yhat = model_fit.predict(len(history), len(history))
    return yhat[0]

def mse(actual, predicted):
    return mean_squared_error(actual, predicted)

def splitData(data, n):
    return data[:-n], data[-n]

def validation(data, n, hyperparams):
    predictions = list()
    train, test = splitData(data, n)
    history = [x for x in train]
    for i in range(len(test)):
        yhat = forecastSARIMA(history, hyperparams)
        predictions.append(yhat)
        history.append(test[i])
    err = mse(test, predictions)
    return err

def testModel(data, n, hyperparams):
    result = None
    key = str(hyperparams)
    result = validation(data, n, hyperparams)
    if result is not None:
        print(' > Model[%s] %.3f' % (key, result))
    return (key, result)

def tuneHyperparams(data, hyperparams, n):
    scores = [testModel(data, n, param) for param in hyperparams]
    scores = [r for r in scores if r[1] != None]
    scores.sort(key=lambda tup: tup[1])
    return scores

def makeHyperparams(seasonal=[0]):
    models = list()
    p_params = [0,1,2]
    d_params = [0,1]
    q_params = [0,1,2]
    t_params = ['n','c','t','ct']
    P_params = [0,1,2]
    D_params = [0,1]
    Q_params = [0,1,2]
    m_params = seasonal
    for p in p_params:
        for d in d_params:
            for q in q_params:
                for t in t_params:
                    for P in P_params:
                        for D in D_params:
                            for Q in Q_params:
                                for m in m_params:
                                    hyperparams = [(p,d,q), (P,D,Q,m), t]
                                    models.append(hyperparams)
    return models

if __name__ == '__main__':
    # Convert data to datetime
    X,y = utils.get_data(2010, 2013, 2013, datafile)
    print(X)
    new_X = pandas.DataFrame(X, columns = ['REGION', 'TOTAL CASES'])
    new_X["TIME"] = X["YEAR"].astype(str) + " " + X["WEEK"].astype(str)
    for i in new_X.shape[0]:
        new_X.at[i, "TIME"] = datetime.strptime(new_X.at[i, "TIME"] + ' 0', "%Y %W %w")
    print(new_X)
    n = 52




[ 0.  0.  1. ... 24.  0.  0.]


ValueError: Shape of passed values is (8424, 1), indices imply (8424, 2)